# Backend Target (V2 API) - Code Laboratory

**Section 4: Running Circuits on IBM Quantum Hardware** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Property/Method | V2 (Current) | V1 (Deprecated) |
|-----------------|--------------|-----------------|
| Get gate set | `backend.target.operation_names` | `backend.configuration().basis_gates` |
| Check support | `target.instruction_supported('cx', qargs=(0,1))` | Manual parsing |
| Coupling map | `target.build_coupling_map()` | `backend.configuration().coupling_map` |
| Num qubits | `backend.num_qubits` | `backend.configuration().n_qubits` |

---

In [ ]:
"""
Qiskit Code Laboratory - Backend Target (V2 API)
================================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
from qiskit import QuantumCircuit, transpile

# IBM Quantum Runtime imports
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeSherbrooke

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def show_target_info(backend, label=""):
    """Display key target information."""
    target = backend.target
    print(f"{label} Target Info:")
    print(f"  Qubits: {target.num_qubits}")
    print(f"  Operations: {target.operation_names}")

backend = FakeManilaV2()
print("✅ Environment ready - using FakeManilaV2 for demonstrations")

---

## `backend.target`

### Access Pattern
```python
target = backend.target    # Attribute, NOT method!
ops = target.operation_names
```

### Key Properties
| Property | Type | Description |
|----------|------|-------------|
| `num_qubits` | `int` | Number of qubits |
| `operation_names` | `set` | All supported operations (gates + measure + reset) |
| `build_coupling_map()` | `CouplingMap` | Qubit connectivity |

### See Also
- [README: Backend V2 API](./README.md#backends)

In [ ]:
# ============================================================
# backend.target - BASIC USAGE
# ============================================================

target = backend.target

print("Target Properties:")
print(f"  num_qubits: {target.num_qubits}")
print(f"  operation_names: {target.operation_names}")

# Build coupling map
coupling = target.build_coupling_map()
print(f"  coupling edges: {coupling.get_edges()[:5]}...")

print("\n✅ target is ATTRIBUTE (not method) - no parentheses!")

In [ ]:
# ============================================================
# operation_names vs basis_gates
# ============================================================

target = backend.target
all_ops = target.operation_names

# Separate gates from non-gate operations
gate_ops = [op for op in all_ops if op not in ['measure', 'reset', 'delay']]
non_gate_ops = [op for op in all_ops if op in ['measure', 'reset', 'delay']]

print("Operation Names Analysis:")
print("=" * 50)
print(f"Gate operations: {gate_ops}")
print(f"Non-gate operations: {non_gate_ops}")

print("\n💡 V2 operation_names includes measure/reset (V1 basis_gates did not)")

---

## `target.instruction_supported()`

### Signature
```python
target.instruction_supported(
    operation_name: str,
    qargs: tuple = None  # (0,) or (0, 1) - MUST be tuple!
) -> bool
```

### Key Points
| Aspect | Correct | Wrong |
|--------|---------|-------|
| Single qubit | `qargs=(0,)` | `qargs=(0)` |
| Two qubits | `qargs=(0, 1)` | `qargs=[0, 1]` |

### See Also
- [README: Gate Support](./README.md#backends)

In [ ]:
# ============================================================
# instruction_supported() - BASIC USAGE
# ============================================================

target = backend.target

print("instruction_supported() Examples:")
print("=" * 50)

# Single-qubit gate
sx_q0 = target.instruction_supported('sx', qargs=(0,))
print(f"SX on qubit 0: {sx_q0}")

# Two-qubit gate (depends on coupling map)
cx_01 = target.instruction_supported('cx', qargs=(0, 1))
cx_02 = target.instruction_supported('cx', qargs=(0, 2))

print(f"CX on (0,1): {cx_01}")
print(f"CX on (0,2): {cx_02}")

# Show coupling map for reference
coupling = target.build_coupling_map()
print(f"\nCoupling edges: {coupling.get_edges()}")

print("\n✅ qargs must be TUPLE: (0,) not (0)")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: V1 vs V2 API
# ============================================================

print("⚠️ TRAP: V1 vs V2 API!")
print("=" * 55)

print("""
❌ V1 API (DEPRECATED - don't use!):
   config = backend.configuration()
   gates = config.basis_gates
   n_qubits = config.n_qubits

✅ V2 API (CURRENT - use this!):
   target = backend.target
   ops = target.operation_names
   n_qubits = backend.num_qubits
""")

# Demonstrate V2
target = backend.target
print(f"V2: target.operation_names = {target.operation_names}")
print(f"V2: backend.num_qubits = {backend.num_qubits}")

print("\n💡 V2 is the Qiskit 2.X certification standard!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: qargs Format
# ============================================================

print("⚠️ TRAP: qargs must be TUPLE!")
print("=" * 55)

target = backend.target

print("""
SINGLE-QUBIT qargs:
  ✅ (0,)   - correct tuple with trailing comma
  ❌ (0)    - just parentheses, NOT a tuple!
  ❌ [0]    - list, not tuple

TWO-QUBIT qargs:
  ✅ (0, 1) - tuple
  ❌ [0, 1] - list, not tuple
""")

# Demo correct usage
correct = target.instruction_supported('sx', qargs=(0,))
print(f"Correct: instruction_supported('sx', qargs=(0,)) = {correct}")

print("\n💡 REMEMBER: qargs=(0,) with trailing comma for single qubit!")

In [ ]:
# ============================================================
# CHALLENGE 1: Check Gate Connectivity
# ============================================================
# Task: Find all valid CX connections for a backend
# Expected: Use instruction_supported to verify connectivity
# ============================================================

def get_cx_connections(backend):
    """
    Find all valid CX gate connections.
    
    Returns:
        List of valid (control, target) pairs
    """
    target = backend.target
    n_qubits = target.num_qubits
    
    valid_cx = []
    for i in range(n_qubits):
        for j in range(n_qubits):
            if i != j and target.instruction_supported('cx', qargs=(i, j)):
                valid_cx.append((i, j))
    
    return valid_cx

# Test on FakeManilaV2
cx_pairs = get_cx_connections(backend)

print("Challenge 1: CX Gate Connectivity")
print("=" * 50)
print(f"Backend: {backend.name}")
print(f"Qubits: {backend.target.num_qubits}")
print(f"Valid CX pairs: {cx_pairs}")

# Verify against coupling map
coupling = backend.target.build_coupling_map()
assert set(cx_pairs) == set(coupling.get_edges())
print("\n✅ Challenge 1 PASSED - Connectivity verified!")

In [ ]:
# ============================================================
# CHALLENGE 2: V1 vs V2 Translation
# ============================================================
# Task: Translate V1 API patterns to V2
# Expected: Demonstrate V2 equivalents
# ============================================================

def v2_backend_info(backend):
    """
    Extract backend info using V2 API only.
    
    Returns:
        Dict with backend information
    """
    target = backend.target
    
    return {
        # V1: backend.configuration().n_qubits → V2:
        'num_qubits': backend.num_qubits,
        
        # V1: backend.configuration().basis_gates → V2:
        'operations': list(target.operation_names),
        
        # V1: backend.configuration().coupling_map → V2:
        'coupling_edges': target.build_coupling_map().get_edges(),
        
        # V2 specific
        'gates_only': [op for op in target.operation_names 
                       if op not in ['measure', 'reset', 'delay']]
    }

# Test
info = v2_backend_info(backend)

print("Challenge 2: V2 API Translation")
print("=" * 50)
print(f"num_qubits: {info['num_qubits']}")
print(f"operations: {info['operations']}")
print(f"gates_only: {info['gates_only']}")

# Verify
assert info['num_qubits'] == 5, "Manila should have 5 qubits"
assert 'cx' in info['gates_only'], "CX should be in gate list"
print("\n✅ Challenge 2 PASSED - V2 API mastered!")

---

## Qubit Properties: T1, T2, Frequency

### Key Concepts
| Property | Definition | Mnemonic |
|----------|------------|----------|
| **T1** | Relaxation time - energy decay from \|1⟩ to \|0⟩ | "Battery life" |
| **T2** | Dephasing time - coherence loss | "Tuning fork" |
| **Frequency** | Resonant frequency for qubit control | "Radio station" |

### Critical Constraint: T2 ≤ 2×T1
T2 can NEVER exceed twice T1 - this is a physical law!

### See Also
- [README: Qubit Properties](./README.md#qubit-properties)

In [ ]:
# ============================================================
# QUBIT PROPERTIES: T1, T2, Frequency - DEMONSTRATION
# ============================================================

from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

# Use a larger backend with realistic properties
backend = FakeSherbrooke()
target = backend.target

print("Qubit Properties (T1, T2, Frequency)")
print("=" * 60)

# Access qubit properties
for qubit_idx in range(min(5, target.num_qubits)):  # First 5 qubits
    props = target.qubit_properties[qubit_idx]
    
    t1 = props.t1 if props.t1 else "N/A"
    t2 = props.t2 if props.t2 else "N/A"
    freq = props.frequency if props.frequency else "N/A"
    
    if isinstance(t1, float):
        t1_us = t1 * 1e6  # Convert to microseconds
        t2_us = t2 * 1e6 if t2 else "N/A"
        freq_ghz = freq / 1e9 if freq else "N/A"
        
        print(f"Qubit {qubit_idx}:")
        print(f"  T1: {t1_us:.1f} μs")
        print(f"  T2: {t2_us:.1f} μs")
        print(f"  Frequency: {freq_ghz:.3f} GHz")
        
        # Verify T2 ≤ 2×T1 constraint
        if isinstance(t2_us, float):
            max_t2 = 2 * t1_us
            print(f"  T2 ≤ 2×T1: {t2_us:.1f} ≤ {max_t2:.1f} ✓")
    else:
        print(f"Qubit {qubit_idx}: Properties not available")

print("\n💡 PHYSICAL CONSTRAINT: T2 ≤ 2×T1 (always true!)")
print("   If T1 = 100μs, max T2 = 200μs")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: T2 ≤ 2×T1 Constraint
# ============================================================

print("⚠️ TRAP: T2 ≤ 2×T1 Physical Constraint")
print("=" * 55)

print("""
EXAM QUESTION FORMAT:
"If T1 = 80μs, what's the maximum possible T2?"

Answer: 160μs (2 × T1)

WHY?
- T2 measures TOTAL decoherence (phase + energy)
- T1 measures ONLY energy relaxation
- Phase loss always accompanies energy loss
- Therefore T2 ≤ 2×T1 is a FUNDAMENTAL LIMIT

❌ WRONG: Assuming any T2 value
✅ CORRECT: Always verify T2 ≤ 2×T1
""")

# Quiz yourself
t1 = 100  # μs
max_t2 = 2 * t1
print(f"Example: If T1 = {t1}μs, then max T2 = {max_t2}μs")

# Circuit depth guideline
print("\n10% Rule for Circuit Depth:")
print(f"  T2 = {max_t2}μs")
print(f"  Circuit time should be < {0.1 * max_t2}μs = {0.1 * max_t2}μs")
print(f"  At ~50ns/gate, that's ~{int(0.1 * max_t2 * 1000 / 50)} gates max depth")

---

## Coupling Maps & SWAP Cost

### Key Concepts
| Concept | Description | Exam Importance |
|---------|-------------|-----------------|
| **Coupling Map** | Graph of direct qubit connections | HIGH |
| **SWAP Gate** | Moves qubit states between positions | HIGH |
| **SWAP = 3 CX** | Each SWAP costs 3 CNOT gates! | **CRITICAL** |

### Routing Overhead
When qubits aren't connected, transpiler inserts SWAPs:
- Each SWAP = 3 CNOTs ≈ 3% error accumulation
- More SWAPs = longer circuits = more decoherence

### See Also
- [README: Coupling Maps](./README.md#coupling-maps)

In [ ]:
# ============================================================
# COUPLING MAPS - Routing & SWAP Cost Demonstration
# ============================================================

from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()
target = backend.target
coupling_map = target.build_coupling_map()

print("Coupling Map Analysis")
print("=" * 60)
print(f"Backend: {backend.name}")
print(f"Qubits: {target.num_qubits}")
print(f"Coupling edges: {coupling_map.get_edges()}")

# Calculate distance between qubits
print("\nQubit Distance Matrix:")
for i in range(target.num_qubits):
    for j in range(target.num_qubits):
        if i < j:
            dist = coupling_map.distance(i, j)
            connected = target.instruction_supported('cx', qargs=(i, j))
            status = "✓ direct" if connected else f"requires {dist-1} SWAPs"
            print(f"  ({i},{j}): distance={dist}, {status}")

# Demo: CX on non-adjacent qubits
print("\n" + "=" * 60)
print("SWAP Cost Demonstration")
print("=" * 60)

# Create circuit with non-adjacent CX
qc = QuantumCircuit(5)
qc.cx(0, 4)  # q0 and q4 are likely not connected

# Transpile and count operations
transpiled = transpile(qc, backend, optimization_level=3, seed_transpiler=42)

print(f"Original circuit: CX(0, 4)")
print(f"Transpiled depth: {transpiled.depth()}")
print(f"Transpiled operations: {transpiled.count_ops()}")

# Calculate SWAP cost
cx_count = transpiled.count_ops().get('cx', 0) + transpiled.count_ops().get('ecr', 0)
print(f"\n2-qubit gate count: {cx_count}")
print(f"  Original: 1 CX")
print(f"  Actual: {cx_count} 2-qubit gates")
print(f"  Routing overhead: {cx_count - 1} extra gates")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: SWAP = 3 CX
# ============================================================

print("⚠️ TRAP: SWAP = 3 CNOT Gates!")
print("=" * 55)

print("""
EXAM CRITICAL: Each SWAP costs 3 CNOTs

SWAP Decomposition:
┌───────────────────────┐
│ SWAP(q0, q1) =        │
│   CX(q0, q1)          │
│   CX(q1, q0)          │
│   CX(q0, q1)          │
└───────────────────────┘

ERROR ACCUMULATION:
  • Each CX ≈ 1% error rate
  • 1 SWAP = 3 CNOTs ≈ 3% error
  • 2 SWAPs = 6 CNOTs ≈ 6% error

WHY IT MATTERS:
  • Non-adjacent CX needs routing
  • Routing uses SWAPs
  • More SWAPs = more errors + longer circuit
""")

# Demonstrate SWAP decomposition
from qiskit import QuantumCircuit

qc_swap = QuantumCircuit(2)
qc_swap.swap(0, 1)

decomposed = qc_swap.decompose()
print("SWAP decomposed:")
print(decomposed.draw())
print(f"Gate count: {decomposed.count_ops()}")

print("\n💡 MEMORIZE: SWAP = 3 CX (routing is expensive!)")

---

## 🎯 Key Takeaways

### Backend Target (V2 API)
- ✅ Use `backend.target` not `backend.configuration()` (deprecated)
- ✅ `target.operation_names` includes gates + measure + reset
- ✅ `target.instruction_supported('cx', qargs=(0,1))` - note tuple format
- ✅ `qargs=(0,)` with trailing comma for single qubit

### Qubit Properties
- ✅ T1 = relaxation time ("battery life")
- ✅ T2 = dephasing time ("tuning fork")
- ✅ **T2 ≤ 2×T1** (physical constraint - always true!)
- ✅ Circuit time should be < 10% of T2

### Coupling Maps & Routing
- ✅ Coupling map shows which qubits can do 2-qubit gates directly
- ✅ **SWAP = 3 CX** (routing is expensive!)
- ✅ Direction matters: `[0,1]` doesn't mean `[1,0]` exists
- ✅ Non-adjacent qubits require SWAP routing

### Mnemonics
```
"T2 ≤ Two Times T1" - Physical constraint
"SWAP = 3 CX" - Routing cost
"TARGET" - Timing, Availability, Reliability, Geometry, Environment, Truth
```

---

## 📚 Resources

- [README.md](./README.md) - Full explanations, traps, mnemonics
- [Section 4 Index](../README.md) - All Section 4 notebooks

---

**Notebook verified with Qiskit 1.x** | Last updated: December 2025